In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.seqsignet_functions import (
    seqsignet_hyperparameter_search
)

In [3]:
output_dir = "therapist_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-01 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-01 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-01 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-01 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-01 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

In [7]:
time_features = ["time_encoding", "timeline_index"]
standardise_method = ["minmax", None]
num_features = len(time_features)
add_time_in_path = True

In [8]:
num_epochs = 100
embedding_dim = 384
dimensions = [15] # [50, 15]
conv_output_channels = [12]#, 10]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3)]#, ([10], 3), ([10], 4)]
lstm_hidden_dim_sizes = [300] # [200, 300]
ffn_hidden_dim_sizes = [[512,512]] # [[64,64],[256,256],[512,512]]
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1]#, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [9]:
shift = 3
window_size = 5
n = 6

In [10]:
anno_mi["transcript_id"]

0          0
1          0
2          0
3          0
4          0
        ... 
13546    131
13547    131
13548    131
13549    131
13550    131
Name: transcript_id, Length: 13551, dtype: int64

In [10]:
size = 20
seqsignet_network_umap_kfold_20, best_seqsignet_network_umap_kfold_20, _, __ = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="main_therapist_behaviour",
    embeddings=sbert_embeddings,
    y_data=y_data_therapist,
    embedding_dim=embedding_dim,
    output_dim=output_dim_therapist,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    conv_output_channels=conv_output_channels,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    time_feature=time_features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    path_indices=therapist_index,
    split_ids=therapist_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_20_kfold.csv


In [12]:
seqsignet_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.625576,0.513856,"[0.7521683820978374, 0.4063745019920319, 0.383...",0.516705,"[0.7340857787810384, 0.45535714285714285, 0.36...",0.515468,"[0.7711643348351909, 0.3669064748201439, 0.408...",None,0.630405,...,False,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.641041,0.482950,"[0.7756458355665129, 0.33561123766135154, 0.33...",0.517749,"[0.7077464788732394, 0.4575569358178054, 0.387...",0.473918,"[0.8579558928147972, 0.26498800959232616, 0.29...",None,0.644932,...,False,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.654721,0.533085,"[0.7814494074749316, 0.4336254462836741, 0.384...",0.543755,"[0.7521386268918623, 0.4727530077848549, 0.406...",0.525968,"[0.8131373013990989, 0.40047961630695444, 0.36...",None,0.654842,...,False,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.656952,0.541935,"[0.7813284638210454, 0.4308420696652012, 0.413...",0.549704,"[0.7531353135313531, 0.49418153607447635, 0.40...",0.539933,"[0.8117144889732038, 0.38189448441247004, 0.42...",None,0.660248,...,False,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.624981,0.535579,"[0.7397124442240952, 0.48173636844891476, 0.38...",0.536865,"[0.7748636717735653, 0.4312796208530806, 0.404...",0.540344,"[0.7076120464785393, 0.5455635491606715, 0.367...",None,0.633221,...,False,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.635539,0.537836,"[0.7513089005235603, 0.46319464649403547, 0.39...",0.540466,"[0.7540004776689754, 0.4499717354437535, 0.417...",0.535999,"[0.7486364714251837, 0.47721822541966424, 0.38...",None,0.641216,...,False,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.633755,0.532209,"[0.7583845063769484, 0.4318630678077683, 0.406...",0.530658,"[0.7553516819571865, 0.47883211678832116, 0.35...",0.541655,"[0.7614417832582404, 0.3932853717026379, 0.470...",None,0.644032,...,False,focal,2,True,5,Conv1d,None,concatenation,64,3
0,None,0.646543,0.545971,"[0.7667899320052487, 0.4715976331360947, 0.399...",0.544965,"[0.771483437349976, 0.4655373831775701, 0.3978...",0.547054,"[0.762153189471188, 0.47781774580335734, 0.401...",None,0.649212,...,False,focal,2,True,5,Conv1d,None,concatenation,64,3
0,None,0.640000,0.541019,"[0.7614335706528296, 0.46992054483541434, 0.39...",0.541019,"[0.7751842751842751, 0.44612068965517243, 0.40...",0.542236,"[0.748162200616552, 0.49640287769784175, 0.382...",None,0.647973,...,False,focal,2,True,5,Conv1d,None,concatenation,64,4
0,None,0.657100,0.541533,"[0.7785542444724481, 0.4558292282430214, 0.390...",0.549415,"[0.7531028368794326, 0.5039941902687001, 0.391...",0.537046,"[0.8057861038653071, 0.41606714628297364, 0.38...",None,0.664752,...,False,focal,2,True,5,Conv1d,None,concatenation,64,4


In [15]:
best_seqsignet_network_umap_kfold_20["f1"].mean()

0.5412757738023036

In [16]:
best_seqsignet_network_umap_kfold_20["precision"].mean()

0.5452170845973715

In [17]:
best_seqsignet_network_umap_kfold_20["recall"].mean()

0.5396411499818743

In [18]:
np.stack(best_seqsignet_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.76999391, 0.46287489, 0.39095853])

In [19]:
np.stack(best_seqsignet_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.76414356, 0.47505744, 0.39645026])

In [20]:
np.stack(best_seqsignet_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.77697415, 0.45623501, 0.38571429])

## GRP

In [25]:
size = 20
seqsignet_network_grp_kfold_20, best_seqsignet_network_grp_kfold_20, _, __ = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="main_therapist_behaviour",
    embeddings=sbert_embeddings,
    y_data=y_data_therapist,
    embedding_dim=embedding_dim,
    output_dim=output_dim_therapist,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    conv_output_channels=conv_output_channels,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    time_feature=time_features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    path_indices=therapist_index,
    split_ids=therapist_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

OSError: [Errno 9] Bad file descriptor: 'best_model_2023-07-14-07-56-21-806023.pkl'

In [ ]:
seqsignet_network_grp_kfold_20

In [ ]:
best_seqsignet_network_grp_kfold_20["f1"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_20["precision"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_20["recall_scores"]).mean(axis=0)